In [1]:
import csv 
import pandas as pd 
import numpy as np
import requests

In [2]:
url = 'https://raw.githubusercontent.com/shaelm/Toronto-Transit-Commision-Delay-Mining/master/datasets/weatherstats_toronto_daily.csv'
df = pd.read_csv(url,sep=",")

In [3]:
df

,date,max_temperature,avg_hourly_temperature,avg_temperature,min_temperature,max_humidex,min_windchill,max_relative_humidity,avg_hourly_relative_humidity,avg_relative_humidity,min_relative_humidity,max_dew_point,avg_hourly_dew_point,avg_dew_point,min_dew_point,max_wind_speed,avg_hourly_wind_speed,avg_wind_speed,min_wind_speed,max_wind_gust,wind_gust_dir_10s,max_pressure_sea,avg_hourly_pressure_sea,avg_pressure_sea,min_pressure_sea,max_pressure_station,avg_hourly_pressure_station,avg_pressure_station,min_pressure_station,max_visibility,avg_hourly_visibility,avg_visibility,min_visibility,max_health_index,avg_hourly_health_index,avg_health_index,min_health_index,heatdegdays,cooldegdays,growdegdays_5,growdegdays_7,growdegdays_10,precipitation,rain,snow,snow_on_ground,sunrise,sunset,sunlight,sunrise_f,sunset_f,min_uv_forecast,max_uv_forecast,min_high_temperature_forecast,max_high_temperature_forecast,min_low_temperature_forecast,max_low_temperature_forecast,solar_radiation,max_cloud_cover_4,avg_hourly_cloud_cover_4,avg_cloud_cover_4,min_cloud_cover_4,max_cloud_cover_8,avg_hourly_cloud_cover_8,avg_cloud_cover_8,min_cloud_cover_8,max_cloud_cover_10,avg_hourly_cloud_cover_10,avg_cloud_cover_10,min_cloud_cover_10
0,2019-11-10,7.4,3.41,3.10,-1.2,NaN,-6.0,90,76.0,78.5,67,1.8,-0.5,-0.4,-2.7,27,17.50,18.5,10,45.0,27.0,102.00,101.42,101.55,101.10,99.83,99.28,99.40,98.97,24100,23100.0,20100,16100,2.5,2.2,2.1,1.8,14.9,0.0,0.0,0.0,0.0,0.6,0.4,0.2,0.0,07:05:00,16:58:00,9.88,7.08,16.97,2.0,2.0,7.0,7.0,-3.0,-3.0,NaN,NaN,NaN,NaN,NaN,8.0,7.7,7.0,6.0,NaN,NaN,NaN,NaN
1,2019-11-09,3.3,0.76,-0.39,-4.1,NaN,-9.0,82,66.0,68.5,55,-2.2,-5.0,-5.2,-8.2,31,22.54,21.0,11,50.0,20.0,102.50,101.82,101.90,101.30,100.32,99.67,99.76,99.21,24100,24100.0,24100,24100,2.9,2.4,2.5,2.2,18.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,07:03:00,16:59:00,9.93,7.05,16.98,1.0,1.0,3.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,8.0,7.0,6.0,4.0,NaN,NaN,NaN,NaN
2,2019-11-08,0.6,-3.00,-2.60,-5.8,NaN,-12.0,80,67.2,66.0,52,-6.0,-8.4,-8.1,-10.2,25,10.96,15.5,6,30.0,4.0,102.90,102.65,102.60,102.30,100.67,100.47,100.42,100.17,24100,24100.0,24100,24100,2.8,2.2,2.3,1.8,20.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,07:02:00,17:00:00,9.97,7.03,17.00,2.0,2.0,0.0,0.0,-5.0,-5.0,NaN,NaN,NaN,NaN,NaN,6.0,2.3,3.0,0.0,NaN,NaN,NaN,NaN
3,2019-11-07,2.6,-0.09,-0.19,-3.0,NaN,-7.0,97,81.3,80.5,64,1.0,-3.0,-2.2,-5.3,20,11.46,11.5,3,32.0,33.0,102.40,102.28,102.25,102.10,100.28,100.13,100.13,99.99,24100,19887.5,13650,3200,2.6,2.4,2.3,2.0,18.2,0.0,0.0,0.0,0.0,3.6,1.6,2.0,1.0,07:01:00,17:02:00,10.02,7.02,17.03,1.0,1.0,2.0,2.0,-5.0,-4.0,NaN,NaN,NaN,NaN,NaN,8.0,6.3,4.5,1.0,NaN,NaN,NaN,NaN
4,2019-11-06,5.2,1.78,1.35,-2.5,NaN,-7.0,92,68.8,68.5,45,1.9,-3.6,-2.5,-7.0,21,14.96,13.5,6,42.0,28.0,102.90,102.60,102.55,102.20,100.78,100.45,100.43,100.08,24100,23166.7,20100,16100,2.8,2.5,2.5,2.2,16.6,0.0,0.0,0.0,0.0,0.6,0.2,0.4,NaN,06:59:00,17:03:00,10.07,6.98,17.05,1.0,2.0,5.0,5.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,8.0,5.4,4.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2013-01-10,6.3,2.85,2.80,-0.7,NaN,NaN,81,68.2,69.0,57,-0.1,-2.5,-2.2,-4.3,30,11.50,15.0,0,32.0,32.0,103.63,103.29,103.27,102.91,101.46,101.13,101.11,100.75,24100,24100.0,24100,24100,3.9,2.9,3.0,2.2,15.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
2496,2013-01-09,6.4,3.40,2.95,-0.5,NaN,-3.0,90,71.5,73.0,56,1.7,-1.5,-1.1,-3.9,44,26.17,25.0,6,59.0,29.0,102.78,101.97,102.10,101.41,100.64,99.83,99.96,99.28,24100,22033.3,18500,12900,3.6,3.0,3.0,2.3,15.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.1,0.5,0.0
2497,2013-01-08,2.7,0.35,0.50,-1.7,NaN,-7.0,90,76.6,78.5,67,-1.8,-3.3,-3.4,-4.9

In [4]:
## Nan count in each column'
df.isnull().sum()

date                            0
max_temperature                 0
avg_hourly_temperature          0
avg_temperature                 0
min_temperature                 0
                             ... 
min_cloud_cover_8             276
max_cloud_cover_10           2341
avg_hourly_cloud_cover_10    2341
avg_cloud_cover_10           2341
min_cloud_cover_10           2341
Length: 70, dtype: int64

In [5]:
## dropping column with Nan values in them  
newdf = df.copy()
for i in newdf.columns:
  if newdf[i].isnull().sum() > 100:
    newdf.drop(columns=[i], inplace=True)

In [6]:
newdf

,date,max_temperature,avg_hourly_temperature,avg_temperature,min_temperature,max_relative_humidity,avg_hourly_relative_humidity,avg_relative_humidity,min_relative_humidity,max_dew_point,avg_hourly_dew_point,avg_dew_point,min_dew_point,max_wind_speed,avg_hourly_wind_speed,avg_wind_speed,min_wind_speed,max_pressure_sea,avg_hourly_pressure_sea,avg_pressure_sea,min_pressure_sea,max_pressure_station,avg_hourly_pressure_station,avg_pressure_station,min_pressure_station,max_visibility,avg_hourly_visibility,avg_visibility,min_visibility,max_health_index,avg_hourly_health_index,avg_health_index,min_health_index,heatdegdays,cooldegdays,growdegdays_5,growdegdays_7,growdegdays_10,precipitation,rain,snow
0,2019-11-10,7.4,3.41,3.10,-1.2,90,76.0,78.5,67,1.8,-0.5,-0.4,-2.7,27,17.50,18.5,10,102.00,101.42,101.55,101.10,99.83,99.28,99.40,98.97,24100,23100.0,20100,16100,2.5,2.2,2.1,1.8,14.9,0.0,0.0,0.0,0.0,0.6,0.4,0.2
1,2019-11-09,3.3,0.76,-0.39,-4.1,82,66.0,68.5,55,-2.2,-5.0,-5.2,-8.2,31,22.54,21.0,11,102.50,101.82,101.90,101.30,100.32,99.67,99.76,99.21,24100,24100.0,24100,24100,2.9,2.4,2.5,2.2,18.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-11-08,0.6,-3.00,-2.60,-5.8,80,67.2,66.0,52,-6.0,-8.4,-8.1,-10.2,25,10.96,15.5,6,102.90,102.65,102.60,102.30,100.67,100.47,100.42,100.17,24100,24100.0,24100,24100,2.8,2.2,2.3,1.8,20.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-11-07,2.6,-0.09,-0.19,-3.0,97,81.3,80.5,64,1.0,-3.0,-2.2,-5.3,20,11.46,11.5,3,102.40,102.28,102.25,102.10,100.28,100.13,100.13,99.99,24100,19887.5,13650,3200,2.6,2.4,2.3,2.0,18.2,0.0,0.0,0.0,0.0,3.6,1.6,2.0
4,2019-11-06,5.2,1.78,1.35,-2.5,92,68.8,68.5,45,1.9,-3.6,-2.5,-7.0,21,14.96,13.5,6,102.90,102.60,102.55,102.20,100.78,100.45,100.43,100.08,24100,23166.7,20100,16100,2.8,2.5,2.5,2.2,16.6,0.0,0.0,0.0,0.0,0.6,0.2,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2013-01-10,6.3,2.85,2.80,-0.7,81,68.2,69.0,57,-0.1,-2.5,-2.2,-4.3,30,11.50,15.0,0,103.63,103.29,103.27,102.91,101.46,101.13,101.11,100.75,24100,24100.0,24100,24100,3.9,2.9,3.0,2.2,15.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2496,2013-01-09,6.4,3.40,2.95,-0.5,90,71.5,73.0,56,1.7,-1.5,-1.1,-3.9,44,26.17,25.0,6,102.78,101.97,102.10,101.41,100.64,99.83,99.96,99.28,24100,22033.3,18500,12900,3.6,3.0,3.0,2.3,15.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2497,2013-01-08,2.7,0.35,0.50,-1.7,90,76.6,78.5,67,-1.8,-3.3,-3.4,-4.9,24,10.54,12.0,0,102.62,102.39,102.33,102.03,100.45,100.22,100.16,99.87,24100,17100.0,19300,14500,4.1,3.5,3.5,3.0,17.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2498,2013-01-07,0.8,-3.15,-3.85,-8.5,83,76.4,76.5,70,-3.4,-6.7,-6.8,-10.3,26,14.33,13.0,0,102.96,102.54,102.51,102.06,100.77,100.36,100.33,99.89,24100,24100.0,24100,24100,3.1,2.6,2.5,1.9,21.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
## dropping columns with no variation (same value)

for i in newdf.columns:
  if len(newdf[i].unique()) == 2:
    newdf.drop(columns=[i], inplace=True)

In [8]:
newdf.head()

,date,max_temperature,avg_hourly_temperature,avg_temperature,min_temperature,max_relative_humidity,avg_hourly_relative_humidity,avg_relative_humidity,min_relative_humidity,max_dew_point,avg_hourly_dew_point,avg_dew_point,min_dew_point,max_wind_speed,avg_hourly_wind_speed,avg_wind_speed,min_wind_speed,max_pressure_sea,avg_hourly_pressure_sea,avg_pressure_sea,min_pressure_sea,max_pressure_station,avg_hourly_pressure_station,avg_pressure_station,min_pressure_station,max_visibility,avg_hourly_visibility,avg_visibility,min_visibility,max_health_index,avg_hourly_health_index,avg_health_index,min_health_index,heatdegdays,cooldegdays,growdegdays_5,growdegdays_7,growdegdays_10,precipitation,rain,snow
0,2019-11-10,7.4,3.41,3.10,-1.2,90,76.0,78.5,67,1.8,-0.5,-0.4,-2.7,27,17.50,18.5,10,102.0,101.42,101.55,101.1,99.83,99.28,99.40,98.97,24100,23100.0,20100,16100,2.5,2.2,2.1,1.8,14.9,0.0,0.0,0.0,0.0,0.6,0.4,0.2
1,2019-11-09,3.3,0.76,-0.39,-4.1,82,66.0,68.5,55,-2.2,-5.0,-5.2,-8.2,31,22.54,21.0,11,102.5,101.82,101.90,101.3,100.32,99.67,99.76,99.21,24100,24100.0,24100,24100,2.9,2.4,2.5,2.2,18.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-11-08,0.6,-3.00,-2.60,-5.8,80,67.2,66.0,52,-6.0,-8.4,-8.1,-10.2,25,10.96,15.5,6,102.9,102.65,102.60,102.3,100.67,100.47,100.42,100.17,24100,24100.0,24100,24100,2.8,2.2,2.3,1.8,20.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-11-07,2.6,-0.09,-0.19,-3.0,97,81.3,80.5,64,1.0,-3.0,-2.2,-5.3,20,11.46,11.5,3,102.4,102.28,102.25,102.1,100.28,100.13,100.13,99.99,24100,19887.5,13650,3200,2.6,2.4,2.3,2.0,18.2,0.0,0.0,0.0,0.0,3.6,1.6,2.0
4,2019-11-06,5.2,1.78,1.35,-2.5,92,68.8,68.5,45,1.9,-3.6,-2.5,-7.0,21,14.96,13.5,6,102.9,102.60,102.55,102.2,100.78,100.45,100.43,100.08,24100,23166.7,20100,16100,2.8,2.5,2.5,2.2,16.6,0.0,0.0,0.0,0.0,0.6,0.2,0.4


In [9]:
newdf.dtypes

date                             object
max_temperature                 float64
avg_hourly_temperature          float64
avg_temperature                 float64
min_temperature                 float64
max_relative_humidity             int64
avg_hourly_relative_humidity    float64
avg_relative_humidity           float64
min_relative_humidity             int64
max_dew_point                   float64
avg_hourly_dew_point            float64
avg_dew_point                   float64
min_dew_point                   float64
max_wind_speed                    int64
avg_hourly_wind_speed           float64
avg_wind_speed                  float64
min_wind_speed                    int64
max_pressure_sea                float64
avg_hourly_pressure_sea         float64
avg_pressure_sea                float64
min_pressure_sea                float64
max_pressure_station            float64
avg_hourly_pressure_station     float64
avg_pressure_station            float64
min_pressure_station            float64


#### for the second section of this notebook i will be using the data at hand to predict if the next day will rain or not. first we convert rain to a binary variable (yes, no), second we will create a new column called rain_tomorrow, this column will be a (yes, no) column aswell inicating if there will be rain the next day 

In [10]:
## first lets fill messing values with mods (most freaquent value in each column)
newdf = newdf.fillna(newdf.mode().iloc[0])

In [11]:
newdf

,date,max_temperature,avg_hourly_temperature,avg_temperature,min_temperature,max_relative_humidity,avg_hourly_relative_humidity,avg_relative_humidity,min_relative_humidity,max_dew_point,avg_hourly_dew_point,avg_dew_point,min_dew_point,max_wind_speed,avg_hourly_wind_speed,avg_wind_speed,min_wind_speed,max_pressure_sea,avg_hourly_pressure_sea,avg_pressure_sea,min_pressure_sea,max_pressure_station,avg_hourly_pressure_station,avg_pressure_station,min_pressure_station,max_visibility,avg_hourly_visibility,avg_visibility,min_visibility,max_health_index,avg_hourly_health_index,avg_health_index,min_health_index,heatdegdays,cooldegdays,growdegdays_5,growdegdays_7,growdegdays_10,precipitation,rain,snow
0,2019-11-10,7.4,3.41,3.10,-1.2,90,76.0,78.5,67,1.8,-0.5,-0.4,-2.7,27,17.50,18.5,10,102.00,101.42,101.55,101.10,99.83,99.28,99.40,98.97,24100,23100.0,20100,16100,2.5,2.2,2.1,1.8,14.9,0.0,0.0,0.0,0.0,0.6,0.4,0.2
1,2019-11-09,3.3,0.76,-0.39,-4.1,82,66.0,68.5,55,-2.2,-5.0,-5.2,-8.2,31,22.54,21.0,11,102.50,101.82,101.90,101.30,100.32,99.67,99.76,99.21,24100,24100.0,24100,24100,2.9,2.4,2.5,2.2,18.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-11-08,0.6,-3.00,-2.60,-5.8,80,67.2,66.0,52,-6.0,-8.4,-8.1,-10.2,25,10.96,15.5,6,102.90,102.65,102.60,102.30,100.67,100.47,100.42,100.17,24100,24100.0,24100,24100,2.8,2.2,2.3,1.8,20.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-11-07,2.6,-0.09,-0.19,-3.0,97,81.3,80.5,64,1.0,-3.0,-2.2,-5.3,20,11.46,11.5,3,102.40,102.28,102.25,102.10,100.28,100.13,100.13,99.99,24100,19887.5,13650,3200,2.6,2.4,2.3,2.0,18.2,0.0,0.0,0.0,0.0,3.6,1.6,2.0
4,2019-11-06,5.2,1.78,1.35,-2.5,92,68.8,68.5,45,1.9,-3.6,-2.5,-7.0,21,14.96,13.5,6,102.90,102.60,102.55,102.20,100.78,100.45,100.43,100.08,24100,23166.7,20100,16100,2.8,2.5,2.5,2.2,16.6,0.0,0.0,0.0,0.0,0.6,0.2,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2013-01-10,6.3,2.85,2.80,-0.7,81,68.2,69.0,57,-0.1,-2.5,-2.2,-4.3,30,11.50,15.0,0,103.63,103.29,103.27,102.91,101.46,101.13,101.11,100.75,24100,24100.0,24100,24100,3.9,2.9,3.0,2.2,15.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2496,2013-01-09,6.4,3.40,2.95,-0.5,90,71.5,73.0,56,1.7,-1.5,-1.1,-3.9,44,26.17,25.0,6,102.78,101.97,102.10,101.41,100.64,99.83,99.96,99.28,24100,22033.3,18500,12900,3.6,3.0,3.0,2.3,15.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2497,2013-01-08,2.7,0.35,0.50,-1.7,90,76.6,78.5,67,-1.8,-3.3,-3.4,-4.9,24,10.54,12.0,0,102.62,102.39,102.33,102.03,100.45,100.22,100.16,99.87,24100,17100.0,19300,14500,4.1,3.5,3.5,3.0,17.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2498,2013-01-07,0.8,-3.15,-3.85,-8.5,83,76.4,76.5,70,-3.4,-6.7,-6.8,-10.3,26,14.33,13.0,0,102.96,102.54,102.51,102.06,100.77,100.36,100.33,99.89,24100,24100.0,24100,24100,3.1,2.6,2.5,1.9,21.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
## converting rain to 1/0 categorical column (1 means rain, 0 means no rain)

newdf['rain'] = (newdf['rain'].values > 0.0)

In [13]:
newdf['rain'] = newdf['rain'].apply(lambda x: 1 if x == True else 0)

In [14]:
newdf['rain']

0       1
1       0
2       0
3       1
4       1
       ..
2495    0
2496    0
2497    0
2498    0
2499    0
Name: rain, Length: 2500, dtype: int64

In [15]:
newdf['rain_tomorrow'] = newdf['rain'].shift(1)

In [21]:
newdf['rain_tomorrow'][0] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
newdf

,max_temperature,avg_hourly_temperature,avg_temperature,min_temperature,max_relative_humidity,avg_hourly_relative_humidity,avg_relative_humidity,min_relative_humidity,max_dew_point,avg_hourly_dew_point,avg_dew_point,min_dew_point,max_wind_speed,avg_hourly_wind_speed,avg_wind_speed,min_wind_speed,max_pressure_sea,avg_hourly_pressure_sea,avg_pressure_sea,min_pressure_sea,max_pressure_station,avg_hourly_pressure_station,avg_pressure_station,min_pressure_station,max_visibility,avg_hourly_visibility,avg_visibility,min_visibility,max_health_index,avg_hourly_health_index,avg_health_index,min_health_index,heatdegdays,cooldegdays,growdegdays_5,growdegdays_7,growdegdays_10,precipitation,rain,snow,rain_tomorrow,year,month,day
0,7.4,3.41,3.10,-1.2,90,76.0,78.5,67,1.8,-0.5,-0.4,-2.7,27,17.50,18.5,10,102.00,101.42,101.55,101.10,99.83,99.28,99.40,98.97,24100,23100.0,20100,16100,2.5,2.2,2.1,1.8,14.9,0.0,0.0,0.0,0.0,0.6,1,0.2,0,2019,11,10
1,3.3,0.76,-0.39,-4.1,82,66.0,68.5,55,-2.2,-5.0,-5.2,-8.2,31,22.54,21.0,11,102.50,101.82,101.90,101.30,100.32,99.67,99.76,99.21,24100,24100.0,24100,24100,2.9,2.4,2.5,2.2,18.4,0.0,0.0,0.0,0.0,0.0,0,0.0,1,2019,11,9
2,0.6,-3.00,-2.60,-5.8,80,67.2,66.0,52,-6.0,-8.4,-8.1,-10.2,25,10.96,15.5,6,102.90,102.65,102.60,102.30,100.67,100.47,100.42,100.17,24100,24100.0,24100,24100,2.8,2.2,2.3,1.8,20.6,0.0,0.0,0.0,0.0,0.0,0,0.0,0,2019,11,8
3,2.6,-0.09,-0.19,-3.0,97,81.3,80.5,64,1.0,-3.0,-2.2,-5.3,20,11.46,11.5,3,102.40,102.28,102.25,102.10,100.28,100.13,100.13,99.99,24100,19887.5,13650,3200,2.6,2.4,2.3,2.0,18.2,0.0,0.0,0.0,0.0,3.6,1,2.0,0,2019,11,7
4,5.2,1.78,1.35,-2.5,92,68.8,68.5,45,1.9,-3.6,-2.5,-7.0,21,14.96,13.5,6,102.90,102.60,102.55,102.20,100.78,100.45,100.43,100.08,24100,23166.7,20100,16100,2.8,2.5,2.5,2.2,16.6,0.0,0.0,0.0,0.0,0.6,1,0.4,1,2019,11,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,6.3,2.85,2.80,-0.7,81,68.2,69.0,57,-0.1,-2.5,-2.2,-4.3,30,11.50,15.0,0,103.63,103.29,103.27,102.91,101.46,101.13,101.11,100.75,24100,24100.0,24100,24100,3.9,2.9,3.0,2.2,15.2,0.0,0.0,0.0,0.0,0.0,0,0.0,1,2013,1,10
2496,6.4,3.40,2.95,-0.5,90,71.5,73.0,56,1.7,-1.5,-1.1,-3.9,44,26.17,25.0,6,102.78,101.97,102.10,101.41,100.64,99.83,99.96,99.28,24100,22033.3,18500,12900,3.6,3.0,3.0,2.3,15.1,0.0,0.0,0.0,0.0,0.0,0,0.0,0,2013,1,9
2497,2.7,0.35,0.50,-1.7,90,76.6,78.5,67,-1.8,-3.3,-3.4,-4.9,24,10.54,12.0,0,102.62,102.39,102.33,102.03,100.45,100.22,100.16,99.87,24100,17100.0,19300,14500,4.1,3.5,3.5,3.0,17.5,0.0,0.0,0.0,0.0,0.0,0,0.0,0,2013,1,8
2498,0.8,-3.15,-3.85,-8.5,83,76.4,76.5,70,-3.4,-6.7,-6.8,-10.3,26,14.33,13.0,0,102.96,102.54,102.51,102.06,100.77,100.36,100.33,99.89,24100,24100.0,24100,24100,3.1,2.6,2.5,1.9,21.9,0.0,0.0,0.0,0.0,0.0,0,0.0,0,2013,1,7


In [18]:
## spliting the date column into (year month day) separate columns
newdf['date'] = pd.to_datetime(df['date'])
newdf['year'] = newdf['date'].dt.year.astype('int')
newdf['month'] = newdf['date'].dt.month.astype('int')
newdf['day'] = newdf['date'].dt.day.astype('int')

In [19]:
newdf.drop(['date'], 1, inplace=True)

In [20]:
cols = list(newdf.columns)
cols.remove('rain_tomorrow')

#### now we have our target, let's build a machine learning model to predict if there will be rain the next day

**data preparation**

In [23]:
from sklearn.model_selection import train_test_split
X = newdf.drop(['rain_tomorrow'], 1)
y = newdf['rain_tomorrow']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [24]:
X_train.shape, X_test.shape

((2000, 43), (500, 43))

In [25]:
### now we scaler the features of (normalize the data)
from sklearn.preprocessing import StandardScaler

enc = StandardScaler()
X_train = enc.fit_transform(X_train)
X_test = enc.transform(X_test)

In [26]:
X_train

array([[-1.66214347, -2.06054013, -1.98471058, ...,  1.54065995,
        -1.60428699,  1.38678441],
       [ 0.65937536,  0.49698626,  0.55704532, ..., -0.49187297,
         1.06334957, -0.98842745],
       [ 1.53847991,  1.14711185,  1.2651059 , ...,  1.54065995,
         1.06334957, -1.66705941],
       ...,
       [ 0.94956521,  0.99806759,  1.06085766, ...,  0.52439349,
         1.06334957, -0.87532212],
       [ 0.76179531,  0.92766018,  0.92015331, ...,  0.52439349,
         0.17413739,  1.38678441],
       [ 0.83861027,  0.59939705,  0.54342877, ..., -0.49187297,
        -0.41867074, -1.55395408]])

In [28]:
X_train = pd.DataFrame(X_train, columns=[cols])
X_test = pd.DataFrame(X_test, columns=[cols])
X_train.head()

,max_temperature,avg_hourly_temperature,avg_temperature,min_temperature,max_relative_humidity,avg_hourly_relative_humidity,avg_relative_humidity,min_relative_humidity,max_dew_point,avg_hourly_dew_point,avg_dew_point,min_dew_point,max_wind_speed,avg_hourly_wind_speed,avg_wind_speed,min_wind_speed,max_pressure_sea,avg_hourly_pressure_sea,avg_pressure_sea,min_pressure_sea,max_pressure_station,avg_hourly_pressure_station,avg_pressure_station,min_pressure_station,max_visibility,avg_hourly_visibility,avg_visibility,min_visibility,max_health_index,avg_hourly_health_index,avg_health_index,min_health_index,heatdegdays,cooldegdays,growdegdays_5,growdegdays_7,growdegdays_10,precipitation,rain,snow,year,month,day
0,-1.662143,-2.060540,-1.984711,-2.297062,-0.011135,0.220205,0.243753,0.378620,-1.447540,-1.984607,-1.898780,-2.273469,0.722314,0.378751,0.787236,0.663076,0.404210,-0.093345,-0.406553,-1.099614,0.234947,-0.237535,-0.539383,-1.168474,-0.0416,-2.752132,-1.639488,-1.800124,0.007591,0.163855,0.087690,0.173679,2.137416,-0.495954,-0.968884,-0.899357,-0.802845,3.251952,-0.689938,17.016225,1.540660,-1.604287,1.386784
1,0.659375,0.496986,0.557045,0.430495,0.499644,0.480223,0.104981,-0.184541,0.575419,0.623355,0.633061,0.680323,0.020600,-0.375301,-0.055670,-0.201430,0.122206,0.168144,0.308857,0.443857,0.160971,0.205081,0.380762,0.520568,-0.0416,0.602490,0.802856,0.908161,-0.933929,-0.898171,-1.096314,-1.020580,-0.746044,-0.495954,0.439199,0.366173,0.185203,-0.413979,-0.689938,-0.198689,-0.491873,1.063350,-0.988427
2,1.538480,1.147112,1.265106,0.932441,1.316889,1.176398,0.521298,-0.114146,1.498516,1.432395,1.400285,1.274740,0.622069,0.108343,0.710608,0.663076,-0.582803,-0.561272,-0.478094,-0.327878,-0.549197,-0.494537,-0.405813,-0.218388,-0.0416,0.163185,-1.186057,-1.297320,0.125281,-0.594735,-0.356311,-1.020580,-1.024092,1.785734,1.505515,1.555466,1.639311,1.830469,1.449405,-0.198689,1.540660,1.063350,-1.667059
3,-0.202659,-0.161369,-0.255409,-0.308219,0.397488,0.891218,0.845100,1.012176,0.123690,0.071305,0.086413,0.040183,0.421579,0.397293,0.557353,0.663076,0.404210,0.154382,0.308857,0.186612,0.427285,0.162247,0.321398,0.203873,-0.0416,0.456448,-0.021564,-0.006028,-1.757758,-1.960197,-1.688316,-1.532406,0.180783,-0.495954,-0.777494,-0.899357,-0.802845,-0.152127,1.449405,-0.198689,0.016260,1.063350,1.612995
4,-0.270939,-0.326872,-0.318953,-0.365043,1.010422,1.058971,0.983872,0.800991,0.153151,-0.061949,-0.009490,-0.151860,0.521824,-0.631802,0.250841,-0.417557,-1.710818,-1.758615,-1.980455,-1.999972,-1.821582,-1.822385,-2.038328,-1.999799,-0.0416,-0.329739,-1.021173,-1.114482,0.125281,0.770727,0.531692,1.197331,0.242571,-0.495954,-0.886860,-0.899357,-0.802845,0.072318,1.449405,-0.198689,0.016260,-1.604287,1.726100


**Feature Importance**


  -in this section i will use XGboost to see the importance of features and rank them accordingly

In [36]:
from xgboost import XGBRegressor
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
imp = pd.DataFrame(xgb.feature_importances_, columns=['Importance'], index=X_train.columns)
imp['col_names'] = imp.index
imp[imp['Importance'] == 0]

[02:12:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


,Importance,col_names
heatdegdays,0.0,"(heatdegdays,)"
cooldegdays,0.0,"(cooldegdays,)"
growdegdays_5,0.0,"(growdegdays_5,)"
growdegdays_7,0.0,"(growdegdays_7,)"
growdegdays_10,0.0,"(growdegdays_10,)"


In [37]:
## dropping columns that have nostatistical significance on the target 
X_train.drop(['heatdegdays', 'cooldegdays', 'growdegdays_5', 'growdegdays_7', 'growdegdays_10'], axis=1, inplace=True)
X_test.drop(['heatdegdays', 'cooldegdays', 'growdegdays_5', 'growdegdays_7', 'growdegdays_10'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:3889: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [39]:
X_train.head()

,max_temperature,avg_hourly_temperature,avg_temperature,min_temperature,max_relative_humidity,avg_hourly_relative_humidity,avg_relative_humidity,min_relative_humidity,max_dew_point,avg_hourly_dew_point,avg_dew_point,min_dew_point,max_wind_speed,avg_hourly_wind_speed,avg_wind_speed,min_wind_speed,max_pressure_sea,avg_hourly_pressure_sea,avg_pressure_sea,min_pressure_sea,max_pressure_station,avg_hourly_pressure_station,avg_pressure_station,min_pressure_station,max_visibility,avg_hourly_visibility,avg_visibility,min_visibility,max_health_index,avg_hourly_health_index,avg_health_index,min_health_index,precipitation,rain,snow,year,month,day
0,-1.662143,-2.060540,-1.984711,-2.297062,-0.011135,0.220205,0.243753,0.378620,-1.447540,-1.984607,-1.898780,-2.273469,0.722314,0.378751,0.787236,0.663076,0.404210,-0.093345,-0.406553,-1.099614,0.234947,-0.237535,-0.539383,-1.168474,-0.0416,-2.752132,-1.639488,-1.800124,0.007591,0.163855,0.087690,0.173679,3.251952,-0.689938,17.016225,1.540660,-1.604287,1.386784
1,0.659375,0.496986,0.557045,0.430495,0.499644,0.480223,0.104981,-0.184541,0.575419,0.623355,0.633061,0.680323,0.020600,-0.375301,-0.055670,-0.201430,0.122206,0.168144,0.308857,0.443857,0.160971,0.205081,0.380762,0.520568,-0.0416,0.602490,0.802856,0.908161,-0.933929,-0.898171,-1.096314,-1.020580,-0.413979,-0.689938,-0.198689,-0.491873,1.063350,-0.988427
2,1.538480,1.147112,1.265106,0.932441,1.316889,1.176398,0.521298,-0.114146,1.498516,1.432395,1.400285,1.274740,0.622069,0.108343,0.710608,0.663076,-0.582803,-0.561272,-0.478094,-0.327878,-0.549197,-0.494537,-0.405813,-0.218388,-0.0416,0.163185,-1.186057,-1.297320,0.125281,-0.594735,-0.356311,-1.020580,1.830469,1.449405,-0.198689,1.540660,1.063350,-1.667059
3,-0.202659,-0.161369,-0.255409,-0.308219,0.397488,0.891218,0.845100,1.012176,0.123690,0.071305,0.086413,0.040183,0.421579,0.397293,0.557353,0.663076,0.404210,0.154382,0.308857,0.186612,0.427285,0.162247,0.321398,0.203873,-0.0416,0.456448,-0.021564,-0.006028,-1.757758,-1.960197,-1.688316,-1.532406,-0.152127,1.449405,-0.198689,0.016260,1.063350,1.612995
4,-0.270939,-0.326872,-0.318953,-0.365043,1.010422,1.058971,0.983872,0.800991,0.153151,-0.061949,-0.009490,-0.151860,0.521824,-0.631802,0.250841,-0.417557,-1.710818,-1.758615,-1.980455,-1.999972,-1.821582,-1.822385,-2.038328,-1.999799,-0.0416,-0.329739,-1.021173,-1.114482,0.125281,0.770727,0.531692,1.197331,0.072318,1.449405,-0.198689,0.016260,-1.604287,1.726100


**Logestic Regression Mode**

In [47]:
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [48]:
from sklearn.linear_model import LogisticRegression


log = LogisticRegression(solver='liblinear', random_state=0)
log.fit(X_train.values, y_train.values)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [49]:
### prediction results:
y_pred_test = log.predict(X_test)

In [53]:
### becuase logistice regression yield probability for each prediction we can get the indivdual probabilities 
### this line will show the probability for the first 10 prediction of not raining tomorrow
log.predict_proba(X_test)[:,0][:10] 

array([0.53479716, 0.78509158, 0.63690147, 0.52307637, 0.4649999 ,
       0.88879712, 0.68874762, 0.82839546, 0.72643845, 0.87292617])

In [54]:
### the accuracy of this model
from sklearn.metrics import accuracy_score
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_test)))

Model accuracy score: 0.7060


In [55]:
### now we will check for the accuracy of the model on the training set to see if our model is not overfitting 
y_pred_train = log.predict(X_train)
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))

Training-set accuracy score: 0.7035


In [56]:
print('Training set score: {:.4f}'.format(log.score(X_train, y_train)))
print('Test set score: {:.4f}'.format(log.score(X_test, y_test)))

Training set score: 0.7035
Test set score: 0.7060


In [57]:
### it looks like the model is not overfitting
### the model might be underfitting because the test and train set results are very similar
### i will try and use less regulization on the model to see if it fixes the problem 

In [64]:
log2 = LogisticRegression(C=2000, solver='liblinear', random_state=0)
log2.fit(X_train, y_train)

LogisticRegression(C=2000, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [65]:
print('Training set score: {:.4f}'.format(log2.score(X_train, y_train)))
print('Test set score: {:.4f}'.format(log2.score(X_test, y_test)))

Training set score: 0.7110
Test set score: 0.7040


In [66]:
### we can see that after using higher C the training set is has slightly higher accuracy.

**Confusion Matrix**

In [69]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_test)

print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[310  37]
 [110  43]]

True Positives(TP) =  310

True Negatives(TN) =  43

False Positives(FP) =  37

False Negatives(FN) =  110


**classificatoin metrics**

In [70]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.74      0.89      0.81       347
           1       0.54      0.28      0.37       153

    accuracy                           0.71       500
   macro avg       0.64      0.59      0.59       500
weighted avg       0.68      0.71      0.67       500



### XGBOOST CLASSIFIER 

we saw how logestic regression perfomerd on the data now lets see how well XGB will perform

In [71]:
import xgboost as xgb
params_xgb ={'n_estimators': 500,
            'max_depth': 16}
model_xgb = xgb.XGBClassifier(**params_xgb)
model_xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=16,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [73]:
y_pred = model_xgb.predict(X_test)

In [75]:
accuracy = accuracy_score(y_test, y_pred)
print('XBG accuracy:', accuracy)

XBG accuracy: 0.698


In [76]:
print('Training set score: {:.4f}'.format(model_xgb.score(X_train, y_train)))
print('Test set score: {:.4f}'.format(model_xgb.score(X_test, y_test)))

Training set score: 1.0000
Test set score: 0.6980


In [77]:
### from the results above we can see that XGB is Overfitting the training set
### i will reduce the number of estimators and max depth 
params_xgb ={'n_estimators': 100,
            'max_depth': 8}
model_xgb = xgb.XGBClassifier(**params_xgb)
model_xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [78]:
print('Training set score: {:.4f}'.format(model_xgb.score(X_train, y_train)))
print('Test set score: {:.4f}'.format(model_xgb.score(X_test, y_test)))

Training set score: 0.9990
Test set score: 0.7120


In [104]:
### we can see model improved quite alot on test set and is performing worse on training set. great, now we need to add more regulizatin
params_xgb ={'n_estimators': 10,
            'max_depth': 10,
            'alpha': 6,
            'gamma': 1}
model_xgb = xgb.XGBClassifier(**params_xgb)
model_xgb.fit(X_train, y_train)

XGBClassifier(alpha=6, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=1,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=10, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [105]:
print('Training set score: {:.4f}'.format(model_xgb.score(X_train, y_train)))
print('Test set score: {:.4f}'.format(model_xgb.score(X_test, y_test)))

Training set score: 0.9480
Test set score: 0.6900


In [ ]:
### we can see by adding more regulization training accuracy is going down but test accuracy is not going up

In [106]:
cm = confusion_matrix(y_test, y_pred_test)

print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[310  37]
 [110  43]]

True Positives(TP) =  310

True Negatives(TN) =  43

False Positives(FP) =  37

False Negatives(FN) =  110


**Random Forrest**

In [122]:
from sklearn.ensemble import RandomForestClassifier

params_rf = {'max_depth': 10,
             'min_samples_leaf': 1,
             'min_samples_split': 2,
             'n_estimators': 40,
             'random_state': 12345}
rf = RandomForestClassifier(**params_rf)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

In [123]:
print('Training set score: {:.4f}'.format(rf.score(X_train, y_train)))
print('Test set score: {:.4f}'.format(rf.score(X_test, y_test)))

Training set score: 0.9325
Test set score: 0.7160


In [125]:
y_pred_test = rf.predict(X_test)
cm = confusion_matrix(y_test, y_pred_test)

print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[310  37]
 [105  48]]

True Positives(TP) =  310

True Negatives(TN) =  48

False Positives(FP) =  37

False Negatives(FN) =  105


**Lgistic Regression with cross validation**

In [130]:
from sklearn.model_selection import cross_val_score
log_cross = LogisticRegression(solver='liblinear')
score = cross_val_score(log_cross, X_train, y_train, cv=5, scoring='accuracy')

In [133]:
score[0]

0.725

In [134]:
### we can see we increased the accuracy score abit with cross validation 